# Eventio & Quadrature Decoders on ESP32

Features to add to ESP32 MicroPython port:

* `ReadyQueue`, `WaitQueue`: simple extensions of `deque`, `utimeq` (see below)
* `timer.Chronometer`: esp32 has monotonic time in us (see below)
* `machine.Decoder`: quadrature decoder, port from Lobo
* `deepsleep` with state retention: use normal sleep

In [1]:
%connect "/dev/cu.SLAB_USBtoUART"

Connected on /dev/cu.SLAB_USBtoUART


## Time / Chronometer

ESP32 offers `int64_t esp_timer_get_time()`

*Get time in microseconds since boot.*

See `ports/esp32/mphalport.c` for examples.

In [1]:
from collections import deque

print(dir(deque))

['__class__', '__name__', 'append', 'popleft']


In [1]:
from utimeq import utimeq

print(dir(utimeq))

['__class__', '__name__', 'peektime', 'pop', 'push']


## Ready Queue

In [1]:
from collections import deque

class ReadyQueue(deque):
    
    def __init__(self, length):
        self.maxlen = length
        super().__init__((), length)
    
    def append(self, item):
        if len(self) >= self.maxlen:
            raise IndexError("waitq is full")
        return super().append(item)

readyq = ReadyQueue(3)

def state(rq, msg):
    print("{:15s}  len = {:2d}  empty = {}".format(msg, len(rq), not bool(rq)))
    
state(readyq, "empty")
readyq.append("first")
readyq.append("second")
readyq.append("third")
state(readyq, "3 items")
print("pop", readyq.popleft())
print("pop", readyq.popleft())
print("pop", readyq.popleft())
try:
    print("pop", readyq.popleft())
    assert True, "popping empty queue"
except IndexError:
    pass
state(readyq, "empty")
readyq.append("first")
readyq.append("second")
readyq.append("third")
try:
    readyq.append("fourth")
    assert True, "appending to full readyq"
except:
    pass
print("pop", readyq.popleft())
print("pop", readyq.popleft())
print("pop", readyq.popleft())


empty            len =  0  empty = True
3 items          len =  3  empty = False
pop first
pop second
pop third
empty            len =  0  empty = True
pop first
pop second
pop third


## Wait Queue

In [1]:
from utimeq import utimeq

class WaitQueue(utimeq):
    
    def __init__(self, length):
        print("init")
        super().__init__(length)
    
    def push(self, time, item):
        super().push(time, item, None)
    
    def pop(self):
        r = [None, None, None]
        super().pop(r)
        return r[1]
    
waitq = WaitQueue(3)

state(waitq, "empty")
waitq.push(3, "t 3")
waitq.push(7, "t 7")
waitq.push(1, "t 1")
try:
    waitq.push(4, "t 4")
    assert True, "push full waitq"
except IndexError:
    pass

state(waitq, "3 items")
print("peektime", waitq.peektime())
print("pop     ", waitq.pop())
print("pop     ", waitq.pop())
print("pop     ", waitq.pop())

try:
    print("pop     ", waitq.pop())
    assert True, "pop empty waitq"
except:
    pass


init
empty            len =  0  empty = True
3 items          len =  3  empty = False
peektime 1
pop      t 1
pop      t 3
pop      t 7


In [1]:
import time

print("us ", time.ticks_us())
print("ms ", time.ticks_ms())
print("cpu", time.ticks_cpu())


us  308397246
ms  5677106
cpu 1002022201
